In [24]:
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [25]:
im1 = io.imread('../Images/1/Goldhill.bmp')
print(im1.shape)

(512, 512)


In [26]:
def downsample_remove_row_and_col(img , factor = 2 ):
    return np.copy(img[::factor , ::factor])

def downsample_using_averaging(img):
    new_height = int(img.shape[0]/2)
    new_width = int(img.shape[1]/2)
    result = np.zeros([new_height, new_width], dtype=np.uint8)
    for i in range(new_height):
        for j in range(new_width):
            result[i,j] = int((img[i*2,j*2].astype(np.uint16) +\
                               img[i*2,j*2+1].astype(np.uint16) +\
                               img[i*2+1,j*2].astype(np.uint16) +\
                               img[i*2+1,j*2+1].astype(np.uint16))/4)
    return result


In [27]:
def upsample_with_replication(img):
    return np.repeat(np.repeat(img, 2, axis=1), 2, axis=0)

def bilinear_interpolation(x, y, points):

    points = sorted(points)    # sort points by x, then by y
    (x1, y1, q11), (_x1, y2, q12), (x2, _y1, q21), (_x2, _y2, q22) = points

    if x1 != _x1 or x2 != _x2 or y1 != _y1 or y2 != _y2:
        raise ValueError('points do not form a rectangle')
    if not x1 <= x <= x2 or not y1 <= y <= y2:
        raise ValueError('(x, y) not within the rectangle')

    return (q11 * (x2 - x) * (y2 - y) +
            q21 * (x - x1) * (y2 - y) +
            q12 * (x2 - x) * (y - y1) +
            q22 * (x - x1) * (y - y1)
           ) / ((x2 - x1) * (y2 - y1) + 0.0)

def upsample_with_bilinear(img):
    def upsample_horizontally(img):
        upsampled_horizontally = np.repeat(img, 2, axis=1)
        for i in range(0,upsampled_horizontally.shape[0]-1):
            for j in range(1,upsampled_horizontally.shape[1]-2,2):
                upper_left = (i-1 , j-1 ,upsampled_horizontally[i-1,j-1])
                lower_right = (i+1 , j+1 ,upsampled_horizontally[i+1,j+1])
                upper_right = (i-1 , j+1 ,upsampled_horizontally[i-1,j+1])
                lower_left = (i+1 , j-1 ,upsampled_horizontally[i+1,j-1])

                upsampled_horizontally[i,j] = bilinear_interpolation(
                i,
                j,
                [upper_left,lower_right,upper_right,lower_left]
                )
        return upsampled_horizontally
    def upsample_vertically(img):
        upsampled_vertically = np.repeat(img, 2, axis=0)
        for i in range(1,upsampled_vertically.shape[0]-2 ,2):
            for j in range(0,upsampled_vertically.shape[1]-1):
                upper = (i-1 , j ,upsampled_vertically[i-1,j-1])
                lower_right = (i+1 , j+1 ,upsampled_vertically[i+1,j+1])
                upper_right = (i-1 , j+1 ,upsampled_vertically[i-1,j+1])
                lower = (i+1 , j ,upsampled_vertically[i+1,j-1])

                upsampled_vertically[i,j] = bilinear_interpolation(
                i,
                j,
                [upper,lower_right,upper_right,lower]
                )
        return upsampled_vertically
    
    return upsample_vertically(upsample_horizontally(img))

In [28]:
downsampled_rmv_col = downsample_remove_row_and_col(im1)
img = Image.fromarray(downsampled_rmv_col)
img.save('downsampled_remove_row_and_col.png')

downsampled_averaging = downsample_using_averaging(im1)
img = Image.fromarray(downsampled_averaging)
img.save('downsampled_averaging.png')


upsampled_replication_rmv = upsample_with_replication(downsampled_rmv_col)
img = Image.fromarray(upsampled_replication_rmv)
img.save('upsampled_replication_rmv.png')
                   
                   
upsampled_bilinear_rmv = upsample_with_bilinear(downsampled_rmv_col)
img = Image.fromarray(upsampled_bilinear_rmv)
img.save('upsampled_bilinear_rmv.png')

upsampled_replication_avg = upsample_with_replication(downsampled_averaging)
img = Image.fromarray(upsampled_replication_avg)
img.save('upsampled_replication_avg.png')
                   
                   
upsampled_bilinear_avg = upsample_with_bilinear(downsampled_averaging)
img = Image.fromarray(upsampled_bilinear_avg)
img.save('upsampled_bilinear_avg.png')

In [29]:
mmse_upsampled_replication_rmv = np.square(np.subtract(img,upsampled_replication_rmv)).mean()
mmse_upsampled_bilinear_rmv = np.square(np.subtract(img,upsampled_bilinear_rmv)).mean()
mmse_upsampled_replication_avg = np.square(np.subtract(img,upsampled_replication_avg)).mean()
mmse_upsampled_bilinear_avg = np.square(np.subtract(img,upsampled_bilinear_avg)).mean()

In [30]:
print(mmse_upsampled_replication_rmv)
print(mmse_upsampled_bilinear_rmv)
print(mmse_upsampled_replication_avg)
print(mmse_upsampled_bilinear_avg)

50.707523345947266
22.782073974609375
29.194293975830078
0.0


In [31]:
def upsample_with_average(img): # this function is not related to HW
    upsampled_image = np.zeros([img.shape[0]*2, img.shape[1]*2,img.shape[2]], dtype=np.uint8)

    for i in range(0,upsampled_image.shape[0]):
        for j in range(0,upsampled_image.shape[1]):
            try:
                if (i%2==0) and (j%2==0):
                    upsampled_image[i,j,:] = np.copy(img[int(i/2),int(j/2)])
                elif (i%2==0) and (j%2==1):
                    upsampled_image[i,j,:] = np.copy((img[int(i/2),int(j/2),:].astype(np.uint16)+img[int(i/2),int(j/2)+1,:].astype(np.uint16))/2)
                elif (i%2==1) and (j%2==0):
                    upsampled_image[i,j,:] = (img[int(i/2),int(j/2)].astype(np.uint16)+img[int(i/2)+1,int(j/2)].astype(np.uint16))/2
                elif (i%2==1) and (j%2==1):
                    upsampled_image[i,j,:] = (img[int(i/2),int(j/2)].astype(np.uint16)+ \
                                              img[int(i/2)+1,int(j/2)+1].astype(np.uint16)+ \
                                              img[int(i/2),int(j/2)+1].astype(np.uint16)+ \
                                              img[int(i/2)+1,int(j/2)].astype(np.uint16))/4
            except:
                pass
